# Example 1: Get protein information, run BLAST, and dump the results into a database


In [1]:
%reload_ext autoreload
%autoreload 2
from pyEED.core import ProteinInfo

## Query NCBI

The pyEED library is centered around the `ProteinSequence` object, which integrates available information on protein sequence, corresponding nucleotide sequence, as well as regions and sites within the sequences. The `ProteinSequence` can be initialized directly with a protein sequence accession number.

In [2]:
aldolase = ProteinInfo.from_ncbi("NP_001287541.1")
print(aldolase)

ProteinInfo
├── id = proteininfo0
├── source_id = NP_001287541.1
├── name = aldolase 1, isoform M
├── sequence = MTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVILFHETLYQKADDGTPFAEILKKKGIILGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPEEIALATVQALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLIRPWALTFSYGRALQASVLRAWAGKKENIAAGQNELLKRAKANGDAAQGKYVAGSAGAGSGSLFVANHAY
├── organism
│   └── Organism
│       ├── id = organism0
│       ├── name = Drosophila melanogaster
│       ├── taxonomy_id = taxon:7227
│       ├── domain = Eukaryota
│       ├── kingdom = Metazoa
│       ├── phylum = Arthropoda
│       ├── tax_class = Insecta
│       ├── order = Diptera
│       ├── family = Drosophilidae
│       ├── genus = Drosophila
│       └── species = melanogaster
├── regions
│   └── 0
│       └── ProteinRegion
│           ├── id = proteinregion0
│           ├── name = Glycolytic
│           ├── 

## BLAST search

In [3]:
blast_results = aldolase.pblast(n_hits=100, e_value=1e-50)

🏃🏼‍♀️ Running PBLAST
├── protein name: aldolase 1, isoform M
├── organism: Drosophila melanogaster
├── e-value: 1e-50
└── max hits: 100


Fetching protein sequences: 100it [00:23,  4.24it/s]


In [ ]:
blast_results.append(aldolase)

In [ ]:
v["f"]

## Storing `ProteinSequence`s in a PostgreSQL database



In [4]:
from sdrdm_database import DBConnector

### Setting up a local MySQL database

First, a local MySQL database needs to be setup. Therefore, we run a docker container with a MySQL database. 
If docker is not installed on your system, please follow the instructions on the [docker website](https://docs.docker.com/get-docker/).


In case this notebook is run on a macOS system with a M1 chip, the following command needs to be run in the terminal first:

>```bash
>export DOCKER_DEFAULT_PLATFORM=linux/amd64
>```

Next, navigate to the directory where this notebook is located and run the following command to start the docker container:

>```bash
>docker compose up -d
>```

### Delete contianers

>```    
>docker rm -vf $(docker ps -aq)
>docker rmi -f $(docker images -aq)
>```

### Connect to the PostgreSQL database

In [5]:
import toml

# Establish a connection to the database
db = DBConnector(**toml.load(open("./env.toml")))

🎉 Connected                                                                                        


### Create tables for `ProteinInfo`

In [6]:
db.create_tables(
    model=ProteinInfo,
    markdown_path="/Users/max/Documents/GitHub/pyeed/specifications/data_model.md",
)


🚀 Creating tables for data model ProteinInfo
│
├── Model 'ProteinInfo' already registered. Skipping.
├── Table 'ProteinInfo'. Already exists in database. Skipping.
├── Table 'ProteinInfo_coding_sequence_ref'. Already exists in database. Skipping.
├── Table 'DNARegion_spans'. Already exists in database. Skipping.
├── Table 'ProteinInfo_sites'. Already exists in database. Skipping.
├── Table 'Site_positions'. Already exists in database. Skipping.
├── Table 'ProteinInfo_regions'. Already exists in database. Skipping.
├── Table 'ProteinRegion_spans'. Already exists in database. Skipping.
├── Table 'ProteinInfo_organism'. Already exists in database. Skipping.
│
╰── 🎉 Created all tables for data model ProteinInfo



In [7]:
# See all created table names
db.connection.list_tables()

['DNARegion_spans',
 'ProteinInfo',
 'ProteinInfo_coding_sequence_ref',
 'ProteinInfo_organism',
 'ProteinInfo_regions',
 'ProteinInfo_sites',
 'ProteinRegion_spans',
 'Site_positions',
 '__model_meta__']

### Populate the database with `ProteinSequence`s

In [8]:
# Insert all blast results into the database
db.insert(*blast_results, verbose=True)

Added dataset ProteinInfo (47452444-edfe-42c1-b85c-7ec6ad4dd154)
Added dataset ProteinInfo (ed6f6103-5371-4618-9b73-d56f72de7b98)
Added dataset ProteinInfo (eaa4f927-e724-4cd6-80df-11a1e7fa2649)
Added dataset ProteinInfo (0e4110e3-2990-4be3-ac4b-52912094bf7e)
Added dataset ProteinInfo (0ea2b50c-009e-4e58-b035-390698ceb91a)
Added dataset ProteinInfo (5520301b-e9e5-46a7-b382-fae567c86580)
Added dataset ProteinInfo (e91b6929-2e1f-4c46-abef-725d44c2d1b5)
Added dataset ProteinInfo (4f85e625-d663-4996-9a84-d8a542aefcba)
Added dataset ProteinInfo (29be2ae5-4529-4721-b073-447dcb3e16c0)
Added dataset ProteinInfo (4c1ad7f0-cbf3-4825-9728-913489189cef)
Added dataset ProteinInfo (155e461e-9bfb-4c8e-8a69-63655878bc74)
Added dataset ProteinInfo (db3397d5-eda9-4144-bef4-16bc6d1fc94c)
Added dataset ProteinInfo (69fc5280-5771-4e92-99c0-17297c43c763)
Added dataset ProteinInfo (81a90d8e-a1bd-4a92-8f23-e571a315f274)
Added dataset ProteinInfo (2ecc5cbd-90bf-4ae6-83cc-9030317dee5e)
Added dataset ProteinInfo

KeyboardInterrupt: 

### Look at entries in the database

In [ ]:
db.connection.table("ProteinInfo_organism")

In [ ]:
# Lets filter the blast results for a specific organism
target = "Drosophila melanogaster"

# First, join the ProteinSequence table with the ProteinSequence_organism table
prot_seqs = db.connection.table("ProteinInfo")
organisms = db.connection.table("ProteinInfo_organism")
joined = prot_seqs.join(
    organisms,
    prot_seqs.ProteinInfo_id == organisms.ProteinInfo_id,
    rname="organism_{name}",
)

# Next, filter the joined table for the target organism
filtered = joined.filter(joined.organism_name == target)
filtered

# Finally, we can get the corresponding ProteinSequence objects
results = db.get("ProteinInfo", filtered)
print(len(results))